## Sample Submission

#### Please note, this notebook is a sample notebook which uses only basic linear regression to make the predictions and submit the csv file, for those who are facing diffuculties in making submissions, and understanding the data.

#### While coding, please ensure you also do your work properly with explanation, as after the end of the competition, a google form will be circulated, wherein you'd be required to submit .ipynb files, with proper explanation and your understanding of the data.

In [1]:
import pandas as pd
import numpy as np

# Load the training data (You might have to change the file path based on how you're working)
train_df = pd.read_csv('/kaggle/input/beginners-hypothesis-25/data_train/working/train.csv')

print(train_df.head())

# For this notebook, we'll only train for 1000 video samples

train_df = train_df[:1000]

   video_id element  motion power  speed          video_summary
0         1    Erde  linear   rot    9.6     (3.8147, 31.94809)
1         2   Feuer     shm  grin    9.6   (26.70288, -4.29153)
2         3   Feuer  random  geld    9.6      (3.8147, 8.58307)
3         4    Erde  zigzag  lila    6.9  (-24.79553, -0.95367)
4         5    Erde  linear  lila    9.6    (7.62939, 22.88818)


In [2]:
import cv2 

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (64, 64))
        frames.append(frame)
    cap.release()
    frames = np.array(frames)  # Shape: (20, 64, 64, 3)
    return frames.flatten()  # Flatten to 1D array
    
video_features = []
for idx, row in train_df.iterrows():
    video_path = f"/kaggle/input/beginners-hypothesis-25/data_train/working/{row['video_id']}.mp4"
    video_features.append(process_video(video_path))

video_features = np.array(video_features)  # Shape: (num_samples, 20*64*64*3)

These video features has all the frames of the video stored as a 1D array

In [3]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical attributes using the Label Encoder
def encode_categorical_columns(df, columns):
    label_encoders = {}
    for col in columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
    return df, label_encoders

categorical_columns = ['element', 'motion', 'power']
train_df, label_encoders = encode_categorical_columns(train_df, categorical_columns)

In [4]:
from sklearn.model_selection import train_test_split


y_speed = train_df[['speed']]
y_summary = train_df['video_summary'].apply(lambda x: eval(x))  # Convert string tuples to actual tuples

y_categorical = train_df[categorical_columns]

# Split data for training and validation
X_train_speed, X_val_speed, y_train_speed, y_val_speed = train_test_split(video_features, y_speed, test_size=0.2, random_state=42)
X_train_cat, X_val_cat, y_train_cat, y_val_cat = train_test_split(video_features, y_categorical, test_size=0.2, random_state=42)
X_train_summary, X_val_summary, y_train_summary, y_val_summary = train_test_split(video_features, y_summary, test_size=0.2, random_state=42)

In [5]:
from sklearn.linear_model import LinearRegression


# Training linear regression models
model_continuous = LinearRegression()
model_continuous.fit(X_train_speed, y_train_speed)

# Train separate models for categorical attributes
models_categorical = {}
for col in categorical_columns:
    model = LinearRegression()
    model.fit(X_train_cat, y_train_cat[col])
    models_categorical[col] = model

In [6]:
# Training for video_summary, separate models for x and y here

x_values, y_values = zip(*y_train_summary)

model_summary_x = LinearRegression()
model_summary_x.fit(X_train_summary, x_values)

model_summary_y = LinearRegression()
model_summary_y.fit(X_train_summary, y_values)

LinearRegression()

### Now that the models are trained, we iterate through the test videos, make the predictions and form the submission.csv

In [7]:
import os

test_folder = '/kaggle/input/beginners-hypothesis-25/data_test/test/'

test_features = []
video_ids = []

# The below function finds all the video ids in the test folder and sorts them, 
# and then stores their features in test_features

for video_id in sorted(
    [f for f in os.listdir(test_folder) if f.endswith(".mp4")], key=lambda x: int(x.split('.')[0])):
    video_path = os.path.join(test_folder, video_id)
    video_ids.append(video_id.split('.')[0]) 
    test_features.append(process_video(video_path))

test_features = np.array(test_features)  # Shape: (num_test_samples, 20*64*64*3)

In [8]:
y_test_speed = model_continuous.predict(test_features)
y_test_summary_x = model_summary_x.predict(test_features)
y_test_summary_y = model_summary_y.predict(test_features)

In [9]:
y_categorical = {}

for col in categorical_columns:
    y_categorical[col] = models_categorical[col].predict(test_features)

In [10]:
# As we are doing regression in this notebook, we need to label the numbers back to what they represent using
# inverse label transform, i.e. 1 will get labelled back as circular for example.

decoded_categorical = {}
for col, predictions in y_categorical.items():
    rounded_predictions = np.rint(predictions).astype(int)
    valid_classes = range(len(label_encoders[col].classes_))
    rounded_predictions = np.clip(rounded_predictions, min(valid_classes), max(valid_classes))
    decoded_categorical[col] = label_encoders[col].inverse_transform(rounded_predictions)

In [11]:
speed_predictions = y_test_speed.flatten()  # Ensure the shape is (num_samples,)
summary_x_predictions = y_test_summary_x.flatten()
summary_y_predictions = y_test_summary_y.flatten()

In [12]:
output_df = pd.DataFrame({
    'video_id': video_ids,                      
    'element': decoded_categorical['element'],  
    'motion': decoded_categorical['motion'],
    'power': decoded_categorical['power'],
    'speed': speed_predictions,               
    'video_summary': list(zip(summary_x_predictions, summary_y_predictions)) 
})

In [13]:
output_df.to_csv("submission.csv", index=False)

# This is the final submission file to be submitted

#### For the participants, you need to perform EDA (Exploratory Data Analysis) well, look for patterns in the data, find some ways to remove the noise for better predictions, both in the videos and text.

#### You are advised to understand all the functions used above in detail, as they'll be helpful in coding further.

# All the best